### Imports

In [1]:
import requests
from arcgis.gis import GIS
from arcgis.features import FeatureLayer
import numpy as np
import pandas as pd

In [2]:
# Login for the notebook running in AGOL
#gis = GIS("home")

# Login for the notebook running in Pro locally...
gis = GIS("pro")

### Variables

In [3]:

# The Item ID of the service containing both the geometry layers and the dashboard layer
# dd is "Disaster Declarations"
dd_id = "d37c3c2a6f1c4586baad82828bfc3c59"

# Get the item at this item ID
dd_item = gis.content.get(dd_id)

# Item ID 0 is the output layer displayed in the dashboard
dashboard_layer = dd_item.layers[0]

# Item ID 1 is the input layer used for getting geometries
geometries_layer = dd_item.layers[1]

print(dashboard_layer)

print(geometries_layer)

<FeatureLayer url:"https://services9.arcgis.com/GDVaV4SDJDDBT8gi/arcgis/rest/services/Disaster_Declarations_Summaries_v2/FeatureServer/0">
<FeatureLayer url:"https://services9.arcgis.com/GDVaV4SDJDDBT8gi/arcgis/rest/services/Disaster_Declarations_Summaries_v2/FeatureServer/1">


In [4]:
# I'm thinking I'll need a little dict of the various fields I'll need to compare
# for querying the geometries layer...?
# The key is the field name in the FEMA data;
# the value is the corresponding field in my geometries layer...
# ACTUALLY I need to do that backwards because placeCODE would create duplicate keys...
# SO the key will be in the geometries; the value will be the FEMA data

compare_dict = {
    "State_FIPS": "fipsStateCode", # This will only be checked where region == "Statewide"
    "Full_FIPS": "fipsCountyCode", # Will have to check the type of this before comparison!
    "AIANNHFP1": "placeCode", # For these last three we will need to construct the full AIA..., fipsStateCode + placeCode
    "AIANNHFP2": "placeCode", # No idea why there are three of these...or why the Summaries sometimes use 2 and 3...
    "AIANNHFP3": "placeCode" # What a pain in the @$$...
}

In [5]:
# I believe I will need three different queries for my geometries layer
# I'll need to query to get state geometries, county geometries and tribal area geometries:

# Get the states on this field:
# (States we'll check first; )
state_field = "State_FIPS"

# Get the counties on this field:
county_field = "Full_FIPS"

# Crappily, in assembling the original dashboard layer I realized that
# the Declaration data may match ANY ONE of these tribal fields, so I need to check all three:
tribal_field1 = "AIANNHFP1"
tribal_field2 = "AIANNHFP2"
tribal_field3 = "AIANNHFP3"


### EVERYTHING BELOW HERE IS FOR THE API...

In [6]:
# Within the API URL, filter the records to return only fyDeclared to 2013 or newer.
# US Census GDBs only go back to 2013; before that it's shapefiles only
# and I refuse to touch shapefiles, at least for the scope of this project.

# Hahaha whoops, forgot that the api only returns 1000 records by default...
# sooooo, had to add orderby desc to get the most recent by declarationDate
# 98% sure this is the correct field to sort by; it should really be the field
# by which records are entered into the db?...
api_url = r"https://www.fema.gov/api/open/v2/DisasterDeclarationsSummaries?$filter=fyDeclared ge 2013&$orderby=declarationDate desc"

In [7]:
# Plug in the URL and capture the response obj
response = requests.get(api_url)

# Convert response to JSON
data = response.json()

# Inspect what I got
type(data)

dict

In [ ]:
for k, v in data.items():
    print(f"{k}, {v}")

In [9]:
# Okay so after a little digging I really only need the following (leave out the metadata)
summaries_df = pd.DataFrame(data["DisasterDeclarationsSummaries"])

In [10]:
# Check my years
print(summaries_df["fyDeclared"].unique())

[2025]


In [11]:
summaries_df.head()

,femaDeclarationString,disasterNumber,state,declarationType,declarationDate,fyDeclared,incidentType,declarationTitle,ihProgramDeclared,iaProgramDeclared,...,placeCode,designatedArea,declarationRequestNumber,lastIAFilingDate,incidentId,region,designatedIncidentTypes,lastRefresh,hash,id
0,FM-5612-CA,5612,CA,FM,2025-09-03T00:00:00.000Z,2025,Fire,2-7 FIRE,False,False,...,99009,Calaveras (County),25121,None,2025090301,9,R,2025-09-03T18:41:07.857Z,d017531813b75fc753371c26b246931d48de651e,28a1ba9f-d914-4024-9e75-4a66b5bba092
1,FM-5611-MT,5611,MT,FM,2025-08-26T00:00:00.000Z,2025,Fire,WINDY ROCK FIRE,False,False,...,99077,Powell (County),25119,None,2025082701,8,R,2025-08-28T18:01:23.160Z,29e175a73b969da6864182e703e3cb3f8d0bb32d,41329e57-2046-4196-a63d-902f3e7c923c
2,FM-5610-OR,5610,OR,FM,2025-08-23T00:00:00.000Z,2025,Fire,FLAT FIRE,False,False,...,99031,Jefferson (County),25117,None,2025082301,10,R,2025-08-25T18:21:58.453Z,8b07b29243bdbba511790332bd3fa9cca0fe33fd,f0604c05-113b-449e-8e4a-f3b5076af546
3,FM-5610-OR,5610,OR,FM,2025-08-23T00:00:00.000Z,2025,Fire,FLAT FIRE,False,False,...,99017,Deschutes (County),25117,None,2025082301,10,R,2025-08-25T18:21:58.453Z,c4a190d030807595da90813aabc6ad2175917668,df7cb24f-8e5a-4c1e-923e-4c75c9ec4581
4,FM-5609-HI,5609,HI,FM,2025-08-19T00:00:00.000Z,2025,Fire,KUNIA ROAD FIRE,False,False,...,99003,Honolulu (County),25114,None,2025082001,9,R,2025-08-21T18:22:16.374Z,731df26a647e5a0338177f445bab7a23b6f8d6ed,ffab7fa0-2e69-428d-b4d3-da95ca352c03


### Okay yeah it's a mess in here right now. Early development stages. Pardon the dust.
Need to figure out exactly how this whole thing is going to work...
...below begin the algorithms...

### Things that will have to be done, in no particular order:

* BEFORE I dissolve (or the pd equivalent of dissolve) the summaries to the FEMA Declaration String level, I need to get all the counties / tribal areas associated with that string so I can grab their geometries and actually perform the spatial dissolve on them
* Tricky with the above: If the FEMA Declaration String applies to both counties and tribal areas, these are treated separately

* Once I have the list of entities the FEMA Declaration String is for, grab the geometries for those counties and perform the dissolve on them. That new dissolve geometry will be the geometry applied to the new row written to the output Summaries dataset.

* THEN dissolve (pd equivalent) the actual summaries, ensuring the schema matches that of the target Summaries dataset...tack on the geometry, apply edits.

* Other tricky bits: Obviously we need to check if a given FEMA string already exists within the summaries dataset. I will need to check whether this is as straightforward as it sounds, or whether...hm. I will have to check whether the potential exists for Summary rows with the same FEMA String to be issued across multiple days...e.g., this string for these three counties is issued this day; then another three rows for an additional three counties are added UNDER THE SAME STRING...this will complicate the checking process.

* Yet more tricky bits: The fields from the summaries used to match the data to the appropriate geometries _is different depending on the level of the entity being matched_. 

For example, Statewide declarations will match on state fips, of course. Counties will match on a combination of state + county code (I could calc that field in the df before I begin...?), and tribal entities will match on...wtf will they match on again...I believe it's a concat of the state FIPS and placeCode...yes because the long tribal codes in the geometries are seven chars...I think. 🥴

I could have just gone easy on myself and committed to representing only county-based declaration rows in my map. But does that simplicity accurately reflect the real world? NO. Does it create a more impressive script? NO! Does it get me a job faster? NO!! 😤

In [12]:
# Some code that will have to happen related to the above bullet:

# Check the types of some important fields...if they came in as strings
# then I don't need to covert them; otherwise I do...come on, big money no whammies!!
print(summaries_df["fipsStateCode"].dtype)
print(summaries_df["fipsCountyCode"].dtype)
print(summaries_df["placeCode"].dtype)

object
object
object


In [13]:
# Add to my df the fields I will need for comparison
summaries_df["fipsFullCode"] = summaries_df["fipsStateCode"] + summaries_df["fipsCountyCode"]
summaries_df["fipsTribalCode"] = summaries_df["fipsStateCode"] + summaries_df["placeCode"]

print(summaries_df["fipsFullCode"].head())

print(summaries_df["fipsTribalCode"].head())

0    06009
1    30077
2    41031
3    41017
4    15003
Name: fipsFullCode, dtype: object
0    0699009
1    3099077
2    4199031
3    4199017
4    1599003
Name: fipsTribalCode, dtype: object


* Considering how I'm going to get the data from the API in shape for the comparison etc. I should just add the two additional columns I added manually for the dashboard I made first, COVID and Entity. After I add and calculate them, the comparisons will all be much easier, because I can just reference those fields for processing the data in chunks (i.e. step 1 process statewide, step 2 process counties, step 3 process tribal)

In [14]:
# Good lord I can't remember how to calculate any of these fields with pandas... 🤣😭
# Anyway the first one I need to calc is the COVID field, simple yes/no

summaries_df["COVID-19"] = np.where(summaries_df["declarationTitle"].str.contains("COVID-19"), "Show only COVID-19", "Show only non-COVID-19")


In [15]:
# Okay why am I only getting threeeeeee COVID records with this...
all_the_bugs = summaries_df[summaries_df["COVID-19"] == "Show only COVID-19"][["declarationTitle", "COVID-19"]]

all_the_bugs

,declarationTitle,COVID-19


In [16]:
# How many records do I have...
summaries_df.shape

(1000, 31)

* Right right right; I forgot that the API by default only returns 1000 records. I shouldn't really NEED more records than that, since the script is going to be run once per day. One thousand records should be MORE than enough. But, I now realize I need to do a little footwork to make sure I am just getting the 1000 _most recent_ records...

Okay added a sort order to the api call to only get the most recent records by declarationDate! That should do it...

In [17]:
summaries_df["declarationDate"].head()

0    2025-09-03T00:00:00.000Z
1    2025-08-26T00:00:00.000Z
2    2025-08-23T00:00:00.000Z
3    2025-08-23T00:00:00.000Z
4    2025-08-19T00:00:00.000Z
Name: declarationDate, dtype: object